In [1]:
#n-Analyzer/blob/master/final_results_gender_test.ipynb
# This notebook author: Reza Chu
# Last Editing Date: 31st May 2019

## Python
import os
import random
import sys


## Package
import glob 
import keras
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as py
import plotly.tools as tls
import seaborn as sns
import scipy.io.wavfile
import tensorflow as tf
py.init_notebook_mode(connected=True)


## Keras
from keras import regularizers
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras.callbacks import  History, ReduceLROnPlateau, CSVLogger
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder


## Rest
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
from tqdm import tqdm

input_duration=3

Using TensorFlow backend.


In [2]:
pip install librosa==0.7.2

  Using cached llvmlite-0.35.0-cp36-cp36m-win_amd64.whl (16.0 MB)
  Created wheel for librosa: filename=librosa-0.7.2-py3-none-any.whl size=1612888 sha256=2451a9f5cd0913460157ac97fc67c74984301c5ed1df8dad16600aab89fb459d
  Stored in directory: c:\users\dmsss\appdata\local\pip\cache\wheels\cb\1d\15\a479fa740849128d481333d2f354f97691be3e2c82480a3e00
Successfully built librosa
  Attempting uninstall: librosa
    Found existing installation: librosa 0.6.3
    Uninstalling librosa-0.6.3:
      Successfully uninstalled librosa-0.6.3
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.31.0
    Uninstalling llvmlite-0.31.0:
      Successfully uninstalled llvmlite-0.31.0
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\users\\dmsss\\anaconda3\\envs\\env2\\lib\\site-packages\\~-vmlite\\binding\\llvmlite.dll'
Consider using the `--user` option or check the permissions.



In [2]:
pip install numba==0.45

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\users\\dmsss\\anaconda3\\envs\\env2\\lib\\site-packages\\~umba\\core\\runtime\\_nrt_python.cp36-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




  Attempting uninstall: numba
    Found existing installation: numba 0.52.0
    Uninstalling numba-0.52.0:
      Successfully uninstalled numba-0.52.0


In [3]:
pip install -U llvmlite==0.32.1

  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.35.0
    Uninstalling llvmlite-0.35.0:
      Successfully uninstalled llvmlite-0.35.0
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\users\\dmsss\\anaconda3\\envs\\env2\\lib\\site-packages\\~~vmlite\\binding\\llvmlite.dll'
Consider using the `--user` option or check the permissions.



In [2]:
pip install plotly==3.4.2

  Using cached retrying-1.3.3.tar.gz (10 kB)
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11434 sha256=3266d0a72b5fdb0cd6d745a538998bec095bf545829daa92eaeae7cd6e693e47
  Stored in directory: c:\users\dmsss\appdata\local\pip\cache\wheels\ac\cb\8a\b27bf6323e2f4c462dcbf77d70b7c5e7868a7fbe12871770cf
Successfully built retrying
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install seaborn==0.9.0


  Using cached seaborn-0.9.0-py3-none-any.whl (208 kB)


In [2]:
dir_list = os.listdir('data/')
dir_list.sort()
print (dir_list)

['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17', 'Actor_18', 'Actor_19', 'Actor_20', 'Actor_21', 'Actor_22', 'Actor_23', 'Actor_24']


In [3]:
data_df = pd.DataFrame(columns=['path', 'source', 'actor', 'gender',
                                'intensity', 'statement', 'repetition', 'emotion'])
count = 0
for i in dir_list:
    file_list = os.listdir('data/' + i)
    for f in file_list:
        nm = f.split('.')[0].split('-')
        path = 'data/' + i + '/' + f
        src = int(nm[1])
        actor = int(nm[-1])
        emotion = int(nm[2])
        
        if int(actor)%2 == 0:
            gender = "female"
        else:
            gender = "male"
        
        if nm[3] == '01':
            intensity = 0
        else:
            intensity = 1
        
        if nm[4] == '01':
            statement = 0
        else:
            statement = 1
        
        if nm[5] == '01':
            repeat = 0
        else:
            repeat = 1
            
        data_df.loc[count] = [path, src, actor, gender, intensity, statement, repeat, emotion]
        count += 1

In [4]:

print (len(data_df))
data_df.head()

1012


,path,source,actor,gender,intensity,statement,repetition,emotion
0,data/Actor_01/03-02-01-01-01-01-01.wav,2,1,male,0,0,0,1
1,data/Actor_01/03-02-01-01-01-02-01.wav,2,1,male,0,0,1,1
2,data/Actor_01/03-02-01-01-02-01-01.wav,2,1,male,0,1,0,1
3,data/Actor_01/03-02-01-01-02-02-01.wav,2,1,male,0,1,1,1
4,data/Actor_01/03-02-02-01-01-01-01.wav,2,1,male,0,0,0,2


In [5]:
filename = data_df.path[1021]
print (filename)

samples, sample_rate = librosa.load(filename)
sample_rate, samples

KeyError: 1021